In [18]:
import pandas as pd

In [19]:
def set_as_category(df, col):
    df[col] = df[col].astype("category")
    return df

def refactor_income(df):
    mapping={'H': 1, 'UM': 1, 'LM': 0, 'L': 0}
    df['income'] = df['income'].map(mapping)
    return df

def melt(df, col_name):
    return pd.melt(df, id_vars = 'Country Code', value_vars = range(2014, 2019), var_name = 'year', value_name = col_name)

def rename_fy_year(df):
    for col in df.columns:
        if col[slice(4)].isdigit():
            df.rename(columns = {col:int(col[slice(4)])}, inplace = True)

    return df

def slice_year(df):
    df['year'] = df['year'].str.slice(0, 4)
    return df

def non_num_na(df):
    for i in df.columns[2:]:
        df[i] = pd.to_numeric(df[i], errors = 'coerce')
    return df

In [20]:
tables = ['gdp_pc', 'inflation', 'trade_pct', 'unemployment', 'income', 'total_export', 'export_neighbors', 'import_neighbors']

for table in tables:
    locals()[table] = pd.read_csv("datafiles/" + table + ".csv")\
        .pipe(rename_fy_year)\
        .pipe(melt, col_name = table)\
        .sort_values(by = ['Country Code', 'year'])

In [21]:
trade = pd.merge(total_export, export_neighbors)
trade = pd.merge(trade, import_neighbors)

trade['trade_ratio'] = trade.export_neighbors / trade.import_neighbors
trade['trade_share'] = trade.export_neighbors / trade.total_export
trade = trade.drop(['total_export', 'export_neighbors', 'import_neighbors'], axis = 1)

trade

,Country Code,year,trade_ratio,trade_share
0,ARM,2014,0.294647,0.100549
1,ARM,2015,0.489140,0.132610
2,ARM,2016,0.495953,0.118256
3,ARM,2017,0.480716,0.108976
4,ARM,2018,0.279415,0.069629
...,...,...,...,...
145,ZWE,2014,0.920771,0.713155
146,ZWE,2015,0.921776,0.764243
147,ZWE,2016,1.028783,0.790168
148,ZWE,2017,1.148331,0.753889


In [22]:
landlocked = pd.merge(gdp_pc, trade)
landlocked = pd.merge(landlocked, inflation)
landlocked = pd.merge(landlocked, trade_pct)
landlocked = pd.merge(landlocked, unemployment)
landlocked = pd.merge(landlocked, income)\
    .pipe(set_as_category, col = 'Country Code')\
    .pipe(set_as_category, col = 'year')\
    .pipe(refactor_income)\
    .pipe(set_as_category, col = 'income')\
    .pipe(non_num_na)\
    .fillna(method="ffill")

landlocked

,Country Code,year,gdp_pc,trade_ratio,trade_share,inflation,trade_pct,unemployment,income
0,ARM,2014,3986.231624,0.294647,0.100549,2.981309,75.778867,17.500000,0
1,ARM,2015,3607.289299,0.489140,0.132610,3.731691,71.682074,18.260000,0
2,ARM,2016,3591.828052,0.495953,0.118256,-1.403608,76.077967,17.620001,0
3,ARM,2017,3914.527854,0.480716,0.108976,0.969553,87.202377,17.700001,1
4,ARM,2018,4220.540321,0.279415,0.069629,2.520234,92.473107,18.969999,1
...,...,...,...,...,...,...,...,...,...
145,ZWE,2014,1434.896277,0.920771,0.713155,-0.197785,54.671615,4.770000,0
146,ZWE,2015,1445.069702,0.921776,0.764243,-2.430968,56.748811,4.778000,0
147,ZWE,2016,1464.588957,1.028783,0.790168,-1.543670,51.219025,4.788000,0
148,ZWE,2017,1235.189032,1.148331,0.753889,0.893962,50.029712,4.785000,0


In [23]:
landlocked.to_csv("datafiles/landlocked.csv")